In [ ]:
!pip install tensorflow==1.6.0

     |████████████████████████████████| 45.9MB 85kB/s 
     |████████████████████████████████| 3.1MB 51.9MB/s 
     |████████████████████████████████| 890kB 48.3MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=1f69429f631277569757bc1da9b360a9566920d15959d695581c43bc61ed9e3b
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.5
    Uninstalling bleach-3.1.5:
      Successfully uninstalled bleach-3.1.5
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or 

1.6.0


In [ ]:
def ResNet_block(input, in_channels, out_channels ):
  conv1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input,  tf.random_normal(shape=[3,3,in_channels,out_channels,]), padding="SAME", strides=[1,out_channels//in_channels,out_channels//in_channels,1]),tf.random_normal([out_channels])),name="blockLayer1")
  conv2 = tf.nn.bias_add(tf.nn.conv2d(conv1, tf.random_normal(shape=[3,3,out_channels,out_channels]), strides=[1,1,1,1], padding="SAME", name="blockLayer2"),tf.random_normal([out_channels]))
  conv3 = tf.cond(tf.cast(out_channels//in_channels>1,dtype=tf.bool), lambda: (tf.add(conv2,tf.nn.bias_add(tf.nn.conv2d(input, tf.random_normal(shape=[1,1,in_channels,out_channels]), padding="SAME", strides=[1,out_channels//in_channels,out_channels//in_channels,1]),tf.random_normal([out_channels])))), lambda: (conv2), name="blockLayer3")
  out_layer = tf.nn.relu(conv3,name="blockOutputLayer")
  return out_layer

In [ ]:
with tf.Session() as sess:
  input1 = tf.placeholder(tf.float32, [None, 125, 125, 1], name="inputs")
  layer1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input1, tf.random_normal(shape=[7,7,1,16]), padding="SAME", strides=[1,2,2,1], name="layer1"),tf.random_normal([16])),name="conv1")
  print(layer1.shape)
  maxpool1 = tf.nn.max_pool(layer1, ksize=[1,2,2,1], name="maxpool1",strides=[1,2,2,1],padding="SAME")
  print(maxpool1.shape)

  block1 = ResNet_block(maxpool1, 16, 16)
  print(block1.shape)
  block1 = ResNet_block(block1, 16, 16)
  print(block1.shape)
  block1 = ResNet_block(block1, 16, 16)  
  print(block1.shape)

  block2 = ResNet_block(block1, 16, 32)
  print(block2.shape)

  block3 = ResNet_block(block2, 32, 32)
  print(block3.shape)
  block3 = ResNet_block(block3, 32, 32)
  print(block3.shape)
  block3 = ResNet_block(block3, 32, 32)
  print(block3.shape)

  maxpool2 = tf.nn.max_pool(block3, ksize=[1,16,32,1], strides=[1,2,2,1], padding="SAME", name="maxpool1")  
  print(maxpool2.shape)
  reshape1 = tf.reshape(maxpool2,[-1,8*8*32])
  print(reshape1.shape)
  dense1 = tf.nn.bias_add(tf.matmul(reshape1,tf.random_normal(shape=[8*8*32,1])),tf.random_normal(shape=[1]),name="outputs")
  print(dense1.shape)


(?, 63, 63, 16)
(?, 32, 32, 16)
(?, 32, 32, 16)
(?, 32, 32, 16)
(?, 32, 32, 16)
(?, 16, 16, 32)
(?, 16, 16, 32)
(?, 16, 16, 32)
(?, 16, 16, 32)
(?, 8, 8, 32)
(?, 2048)
(?, 1)


In [ ]:
g = tf.Graph()
with g.as_default():
  input1 = tf.placeholder(tf.float32, [None, 125, 125, 1], name="inputs")
  layer1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input1, tf.random_normal(shape=[7,7,1,16]), padding="SAME", strides=[1,2,2,1], name="layer1"),tf.random_normal([16])),name="conv1")
  print(layer1.shape)
  maxpool1 = tf.nn.max_pool(layer1, ksize=[1,2,2,1], name="maxpool1",strides=[1,2,2,1],padding="SAME")
  print(maxpool1.shape)

  block1 = ResNet_block(maxpool1, 16, 16)
  print(block1.shape)
  block1 = ResNet_block(block1, 16, 16)
  print(block1.shape)
  block1 = ResNet_block(block1, 16, 16)  
  print(block1.shape)

  block2 = ResNet_block(block1, 16, 32)
  print(block2.shape)

  block3 = ResNet_block(block2, 32, 32)
  print(block3.shape)
  block3 = ResNet_block(block3, 32, 32)
  print(block3.shape)
  block3 = ResNet_block(block3, 32, 32)
  print(block3.shape)

  maxpool2 = tf.nn.max_pool(block3, ksize=[1,16,32,1], strides=[1,2,2,1], padding="SAME", name="maxpool1")  
  print(maxpool2.shape)
  reshape1 = tf.reshape(maxpool2,[-1,8*8*32])
  print(reshape1.shape)
  dense1 = tf.nn.bias_add(tf.matmul(reshape1,tf.random_normal(shape=[8*8*32,1])),tf.random_normal(shape=[1]),name="outputs")
  print(dense1.shape)


(?, 63, 63, 16)
(?, 32, 32, 16)
(?, 32, 32, 16)
(?, 32, 32, 16)
(?, 32, 32, 16)
(?, 16, 16, 32)
(?, 16, 16, 32)
(?, 16, 16, 32)
(?, 16, 16, 32)
(?, 8, 8, 32)
(?, 2048)
(?, 1)


In [ ]:
g.get_operations(),g.get_tensor_by_name('outputs:0')

([<tf.Operation 'inputs' type=Placeholder>,
  <tf.Operation 'random_normal/shape' type=Const>,
  <tf.Operation 'random_normal/mean' type=Const>,
  <tf.Operation 'random_normal/stddev' type=Const>,
  <tf.Operation 'random_normal/RandomStandardNormal' type=RandomStandardNormal>,
  <tf.Operation 'random_normal/mul' type=Mul>,
  <tf.Operation 'random_normal' type=Add>,
  <tf.Operation 'layer1' type=Conv2D>,
  <tf.Operation 'random_normal_1/shape' type=Const>,
  <tf.Operation 'random_normal_1/mean' type=Const>,
  <tf.Operation 'random_normal_1/stddev' type=Const>,
  <tf.Operation 'random_normal_1/RandomStandardNormal' type=RandomStandardNormal>,
  <tf.Operation 'random_normal_1/mul' type=Mul>,
  <tf.Operation 'random_normal_1' type=Add>,
  <tf.Operation 'BiasAdd' type=BiasAdd>,
  <tf.Operation 'conv1' type=Relu>,
  <tf.Operation 'maxpool1' type=MaxPool>,
  <tf.Operation 'random_normal_2/shape' type=Const>,
  <tf.Operation 'random_normal_2/mean' type=Const>,
  <tf.Operation 'random_normal_2/

In [ ]:
with tf.Session(graph=g) as sess:
    out=sess.run("outputs:0",feed_dict={"inputs:0":np.random.rand(10,125,125,1)})

In [ ]:
out

array([[-1.1196280e+16],
       [-1.6319422e+16],
       [-1.3355599e+16],
       [-1.2417205e+16],
       [-1.3140401e+16],
       [-1.5486815e+16],
       [-1.3342215e+16],
       [-1.1745529e+16],
       [-1.3813199e+16],
       [-1.5031454e+16]], dtype=float32)

In [ ]:
graph_def = g.as_graph_def()
EXPORT_DIR = './pb_model'
tf.train.write_graph(graph_def, EXPORT_DIR,'ResNet.pb', as_text=False)

'./pb_model/ResNet.pb'